In [40]:
import numpy as np
import pandas as pd
import re
import csv
import os
from parsivar import Normalizer
from nltk import NLTKWordTokenizer

In [59]:
def count_english_words(row):
    if row is None:
        print(row)
        return 10
    words = row.split(' ')
    cnt = 0
    for word in words:
        if re.match('[a-zA-z]+', word):
            cnt += 1
            
    return cnt

In [60]:
def read_data_from_dir(directory='./'):
    files = os.listdir(directory)
    data = []
    for file in files:
        if 'done' not in file: 
            continue
        with open(file, 'r') as txtfile:
            data.append(txtfile.read())

    data = '\n'.join(data).split('\n')
    data = [x.split('\t')[:2] for x in data]
    df = pd.DataFrame(data, columns=['Source', 'Target'])
    return df

In [61]:
MAX_SOURCE_LEN = 30

fa_normalizer = Normalizer()

data = read_data_from_dir()
data['Source'] = data['Source'].str.strip().replace('...', ' ')
data['Target'] = data['Target'].str.strip().replace('...', ' ')
data['Target'] = data['Target'].str.strip().replace('\u200c', ' ')
data['Target'] = data['Target'].map(lambda x: fa_normalizer.normalize(x) if x is not None else None)
data = data[data['Target'].map(lambda x: count_english_words(x) == 0)]
data = data[data['Source'].map(lambda x: len(x.split(' ')) <= MAX_SOURCE_LEN)]
data = data.drop_duplicates(keep='first')
print(len(data))
data = data.dropna()
data

None
None
185931


,Source,Target
0,the current king is only kim myung 's puppet .,پادشاه فعلی فقط عروسک خیمه شب بازی کیم میونگ ا...
1,"how could he ask for consul jang 's help ,",چگونه می‌توانست از کنسول جانگ کمک بخواهد ،
2,when he must know what happened between kim my...,وقتی او باید بداند بین کیم میونگ و کنسول جانگ ...
3,but he said in the letter that his life was in...,اما در نامه گفته است که جانش در خطر است .
4,he would seek help anywhere if his life were i...,اگر جانش در خطر‌بود از هر جا کمک می‌گرفت .
...,...,...
309375,"the police werent going to get here , .",پلیس قرار نبود به اینجا برسد .
309376,so i followed him .,بنابراین من او را دنبال کردم .
309377,"i mean , i imagine , its just a boy .",منظورم این است که تصور می‌کنم این فقط یک پسر ا...
309378,what did the police do with my car .,پلیس با ماشین من چه کرد .


In [62]:
data.to_csv('combined.txt', sep='\t', header=False, index=False, encoding='utf-8')